In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/Touche

 2020
 Initial_retrieval_bm25.ipynb
 relevant_bm25.csv
 relevant_bm25.jsonl
 reranked.csv
 Reranker.ipynb
 s2v_old
 sample_collection_jsonl
't5 base'
 topics-task-2-2020.xml
 topics-task2.xml
 touche2020-task2-relevance-withbaseline.qrels
'Touche _2022.ipynb'
 Touche-21-Task-2
 touche-task2-passages-expanded-with-queries.jsonl
 touche-task2-passages-version-002.jsonl
 Untitled0.ipynb


In [3]:
# installing linux related stuff for pyserini
# removing error "ImportError: No module named '_swigfaiss"
# Reference Link: https://github.com/facebookresearch/faiss/issues/821
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (409 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty

In [ ]:
# installing important packages for analyzing the code.
!pip install jsonlines
!pip install pyserini
!pip install faiss

In [5]:
# import statements
import pandas as pd
import jsonlines
import pyserini
from tqdm import tqdm
from xml.dom import minidom
import re
import nltk
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS
import string
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
# query expansion corpus
qcorpus_list = pd.read_csv("/content/drive/MyDrive/Touche/2020/docs_2020.csv")

In [9]:
# printing demo query
qcorpus_list.head()

,Unnamed: 0,old_id,contents
0,0,clueweb12-0000tw-00-14115,Do Asian-Americans Face Bias in Admissions at ...
1,1,clueweb12-0000tw-00-15738,Big Data Analytics a Key Enabler for Social CR...
2,2,clueweb12-0000tw-00-17905,Best Bets: Fern brings magic to Milford - Delm...
3,3,clueweb12-0000tw-01-02230,Python Data Analysis Library — pandas: Python ...
4,4,clueweb12-0000tw-01-15084,MILLARWORLD WEEK: Mark Millar’s Master Plan | ...


In [10]:
#functions for preprocessing
special_characters=string.punctuation
special_characters=special_characters+'“”’—'
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()


def spl_chars_removal(lst):
  lst1=list()
  for element in lst:
    str=""
    str="".join([i for i in element if i not in special_characters])
    lst1.append(str)
  return lst1

def stopwords_removal_gensim_custom(lst):
  tokens_without_sw = [word for word in lst if not word in STOPWORDS]
  return tokens_without_sw

def stemming(lst):
  stem_text = [porter_stemmer.stem(word) for word in lst]
  return stem_text

def lemmatizer(lst):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in lst]
  return lemm_text

In [11]:
!mkdir sample_collection_jsonl
!mkdir indexes
!mkdir indexes/sample_collection_jsonl

In [16]:
#creating an input file for indexing without preprocessing
output = jsonlines.open('sample_collection_jsonl/documents.jsonl', 'w')
for vals in tqdm(qcorpus_list.iterrows()):
  output.write({'id':vals[1].old_id,'contents': vals[1].contents})

In [17]:
#creating an input file for indexing with preprocessing
output = jsonlines.open('sample_collection_jsonl/documents.jsonl', 'w')
for k in tqdm(qcorpus_list.iterrows()):
  i=k[1].contents
  i=re.sub("[</]query","",i)
  i=i.lower()
  z = i.split(" ")
  z=spl_chars_removal(z)
  z=stopwords_removal_gensim_custom(z)
  z=' '.join(z)
  output.write({'id': k[1].old_id,'contents': z})

33401it [03:14, 171.47it/s]


In [18]:
!head -10 sample_collection_jsonl/documents.jsonl

{"id": "clueweb12-0000tw-00-14115", "contents": "asianamericans face bias admissions elite colleges  nytimescom home page todays paper video popular times topics search nytimescom education world politics education bay area chicago texas ny  region business technology science health sports opinion arts style travel jobs real estate autos february 8 2012 143 pm asianamericans face bias admissions elite colleges daniel e slotnik 608 pm  updated statement princeton added story department educations office civil rights examining complaints thatharvard princeton discriminated asianamerican undergraduate applicants highlighting concern asianamerican parents inquiry reported bloomberg news office civil rights confirmed investigation stemmed individual complaints reported schools august 2011 declined provide details studies suggest asianamerican students need higher standardized test scores white students admitted colleges harvard rejected charges bias harvard college discriminate asianamerica

In [19]:
#index builder
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input sample_collection_jsonl \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 4 \
  --storePositions --storeDocvectors --storeRaw

2022-04-01 10:38:09,008 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2022-04-01 10:38:09,011 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2022-04-01 10:38:09,011 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2022-04-01 10:38:09,012 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: sample_collection_jsonl
2022-04-01 10:38:09,012 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2022-04-01 10:38:09,013 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2022-04-01 10:38:09,014 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 4
2022-04-01 10:38:09,014 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Language: en
2022-04-01 10:38:09,015 INFO  [main] index.IndexCollection

In [20]:
!ls /content/indexes/sample_collection_jsonl

_0.fdm	_0_Lucene80_0.dvd  _0_Lucene84_0.tim  _0.nvm  _0.tvx
_0.fdt	_0_Lucene80_0.dvm  _0_Lucene84_0.tip  _0.si   segments_1
_0.fdx	_0_Lucene84_0.doc  _0_Lucene84_0.tmd  _0.tvd  write.lock
_0.fnm	_0_Lucene84_0.pos  _0.nvd	      _0.tvm


In [21]:
#saving the index files to the drive
!cp -r /content/indexes/sample_collection_jsonl /content/drive/MyDrive/Touche/2020

In [22]:
def parse_xml(path):
  answer_list = []
  xmldoc = minidom.parse(path)
  itemlist = xmldoc.getElementsByTagName('topics')
  topic_list = itemlist[0].getElementsByTagName('topic')
  for topic in topic_list:
    tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
    answer_list.append(tuple_for_add)
  parsed=pd.DataFrame(answer_list, columns=["Number","Title"])
  return parsed

In [23]:
topics=parse_xml("/content/drive/MyDrive/Touche/topics-task-2-2020.xml")
topics.head()

,Number,Title
0,1,\nWhat is the difference between sex and love?\n
1,2,"\nWhich is better, a laptop or a desktop?\n"
2,3,"\nWhich is better, Canon or Nikon?\n"
3,4,\nWhat are the best dish detergents?\n
4,5,\nWhat are the best cities to live in?\n


In [24]:
#creating a dataframe for intial retrieval
from pyserini.search import SimpleSearcher
from pyserini.search.lucene import LuceneSearcher
searcher = LuceneSearcher('/content/indexes/sample_collection_jsonl')
h1=[]
for l,j in tqdm(zip(topics["Title"],topics["Number"])):
  hits = searcher.search(l, k=1000)
  for i in range(len(hits)):
    contents=str.split(hits[i].raw,'"contents" :',)[1]
    contents=contents.replace("}","")
    contents=contents.replace('"',"")
    contents=contents.replace('\n',"")
    h1.append({"title_id":j,"title":l,'doc_id': hits[i].docid,'score': hits[i].score,"content": contents})

  # Value of K determines the number of documents that can be returned.
  print('Number of document matches from the index: '+str(len(hits)))

1it [00:01,  1.38s/it]

Number of document matches from the index: 1000


2it [00:01,  1.22it/s]

Number of document matches from the index: 1000


3it [00:02,  1.45it/s]

...

Number of document matches from the index: 1000


49it [00:29,  1.56it/s]

Number of document matches from the index: 1000


50it [00:30,  1.65it/s]

Number of document matches from the index: 1000


In [25]:
df=pd.DataFrame(h1)
df.head()

,title_id,title,doc_id,score,content
0,1,\nWhat is the difference between sex and love?\n,clueweb12-0818wb-26-13074,7.1866,toofly nyc » hip hop culture toofly nyc clien...
1,1,\nWhat is the difference between sex and love?\n,clueweb12-0010wb-33-27298,7.1624,relational quantum mechanics stanford encyclo...
2,1,\nWhat is the difference between sex and love?\n,clueweb12-0103wb-41-14265,6.9118,lecture series science software engineering...
3,1,\nWhat is the difference between sex and love?\n,clueweb12-1013wb-51-17839,6.4538,ethical buyers guide toy cars trains planes s...
4,1,\nWhat is the difference between sex and love?\n,clueweb12-1400tw-39-23968,6.2936,video video cbr tv movie trailersclips tv tra...


In [26]:
df.to_csv("/content/drive/MyDrive/Touche/2020/relevant_bm25_2020.csv")